Sagnik De
Email : sagnik4de@gmail.com
Emotion Recognition

In [1]:
# from google.colab import drive
# 
# drive.mount('/content/drive/')
# %cd drive/MyDrive/EmotionRecognition

These are the dependencies. Running them will automatically install it.

In [2]:
!pip install deepface
!pip install ultralytics


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


This is for showing the image with the bounding boxes with the emotions detected. This will also the return the modified image so that it can be save in the target directory.

In [6]:
from PIL import ImageFont, ImageDraw, Image


def show_bbox_with_emotion(image: Image, bboxes: dict, do_show=True):
    draw = ImageDraw.Draw(image)
    fnt = ImageFont.truetype("../../deepfaceAndEmotion/GlacialIndifference-Regular.otf", 12)
    for bbox in bboxes:
        x1 = int(bbox['x1'])
        y1 = int(bbox['y1'])
        x2 = int(bbox['x2'])
        y2 = int(bbox['y2'])
        draw.rectangle([x1, y1, x2, y2], outline=(0, 255, 0, 85), width=1)
        emotion = f"{bbox['f_i']}_{bbox['emotion']}"
        draw.text((x1, y1), emotion, font=fnt, fill=(255, 0, 0))
    if do_show:
        image.show()

The function below is used to acquire the emotion of the face defined by the bounding box.

In [7]:

from deepface import DeepFace


def get_emotion(cropped_face):
    analyzed_emotion = DeepFace.analyze(cropped_face, actions=['emotion'], enforce_detection=False)[0]
    return {'emotion': analyzed_emotion['dominant_emotion'],
            'emo_confidence': analyzed_emotion['face_confidence']}

So we want to use the Yolo library and dump the folder with all the images so that processing can be done all in bulk fashion. Here I am returning a dataframe of the inferred results. That can be saved as csv for later analysis. 

In [8]:
import pandas as pd
from ultralytics import YOLO

model = YOLO('yolov8n-face.pt')


def detect_face(path_to_images, path_to_results, limit):
    inference_results = []
    inferences = model(path_to_images,stream=True)
    for image_number, inference in enumerate(inferences):
        if 0 < limit <= image_number:
            break
        image_path = inference.path
        for i, detected_face in enumerate(inference):
            given_xyxy = detected_face.boxes.xyxy[0]
            x1 = int(given_xyxy[0].item())
            y1 = int(given_xyxy[1].item())
            x2 = int(given_xyxy[2].item())
            y2 = int(given_xyxy[3].item())
            h, w = inference.orig_shape
            face_box = inference.orig_img[y1:y2, x1:x2]
            emotional_resp = get_emotion(face_box)
            common_file_data = {'image_path': image_path, 'h': h, 'w': w, 'f_i': i}
            xys = {'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2}
            all_facial_results = {**common_file_data, **xys, **emotional_resp}
            inference_results.append(all_facial_results)
    df = pd.DataFrame(inference_results)
    if path_to_results is not None:
        df.to_csv(path_to_results)
    return df

Store generated images with bounding boxes.

In [33]:
import os


def store_with_bounding_box(csv_loc, target_location):
    df = pd.read_csv(csv_loc)
    file_paths = df['image_path'].unique()
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        print(f'Generating bounding box for {file_name} and saving it to {target_location}')
        filtered = df[df['image_path'].str.contains(file_name)]
        list_faces = list(filtered.T.to_dict().values())
        image = Image.open(file_path).convert('RGB')
        show_bbox_with_emotion(image, list_faces, do_show=False)
        png_name = file_name.replace('.jpg', '.png')
        location = os.path.join(target_location, png_name)
        image.save(location)

The first 100 images will be inferred with the 

In [34]:
detect_face('../../deepfaceAndEmotion/source/', 'recognised_emotions.csv', 100)
store_with_bounding_box('recognised_emotions.csv','target')

image 1/3083 C:\mywork\ml\deepfaceAndEmotion\source\001333d5a0464e2fb454647fb3cf1dce.jpg: 416x640 11 faces, 64.4ms
image 2/3083 C:\mywork\ml\deepfaceAndEmotion\source\00746310ec034c5484f3b998cbfa4795.jpg: 640x640 4 faces, 83.1ms
image 3/3083 C:\mywork\ml\deepfaceAndEmotion\source\014a05e9ae584321a9f473c994dd9818.jpg: 480x640 2 faces, 62.2ms
image 4/3083 C:\mywork\ml\deepfaceAndEmotion\source\0150b34a95e04a2c8d588af9942aec2d.jpg: 544x640 4 faces, 86.2ms
image 5/3083 C:\mywork\ml\deepfaceAndEmotion\source\01537a90201f483c8492876384636764.jpg: 544x640 56 faces, 63.3ms
image 6/3083 C:\mywork\ml\deepfaceAndEmotion\source\022915fab0af4d5dbc74c88214a21ee5.jpg: 384x640 6 faces, 58.7ms
image 7/3083 C:\mywork\ml\deepfaceAndEmotion\source\0272a05589274fedaa11b1c4a439287a.jpg: 544x640 9 faces, 166.0ms
image 8/3083 C:\mywork\ml\deepfaceAndEmotion\source\0275f9a215934751b2b477d509cf3491.jpg: 448x640 9 faces, 59.9ms
image 9/3083 C:\mywork\ml\deepfaceAndEmotion\source\02a9ea1d381743e99e9e0eb95e746767.

The below code is to visualize an already inferred data from the csv file.

In [31]:
def visualize(csv_loc, fileName):
    df = pd.read_csv(csv_loc)
    filtered = df.loc[df['image_path'].str.contains(fileName)]
    list_faces = list(filtered.T.to_dict().values())
    image_loc = list_faces[0]['image_path']
    image = Image.open(image_loc).convert('RGB')
    show_bbox_with_emotion(image, list_faces)

In [32]:
#visualize('recognised_emotions.csv','0884b043075e42be8f0fb6a962777d13')